# <font color='orange'>Create an Elastic Serverless Search Project Programmatically</font>


## This notebook will show you how to:
 - Create an Elastics Serverless Project

[Serverless API Docs](https://www.elastic.co/docs/api/doc/elastic-cloud-serverless/operation/operation-createelasticsearchproject#operation-createelasticsearchproject-body-application-json-optimized_for)

This tutorial assumes you have an Elastic Cloud account.<br>
If not head over to [cloud.elastic.co/trial](https://cloud.elastic.co/trial)

# Notebook Setup
- Install elasticsearch python library
- import required libraries

In [ ]:
!pip install -qq elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.5/960.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
import requests
import getpass
from pprint import pprint
from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ConnectionTimeout
from IPython.display import clear_output
from time import sleep

# Setup Your Cloud API Key

1. Generate your secret API key at https://cloud.elastic.co/account/keys
2. Run the next cell to store your API Key

In [ ]:
api_key = getpass.getpass("Enter your API key: ")

print("API key successfully entered!")

Enter your API key: ··········
API key successfully entered!


## Create Elasticsearch project


Configure:
- name of project - <font color='orange'>`name`</font>
- cloud region - <font color='orange'>`region_id`</font>
- headers

Then create the project!

In [ ]:
url = "https://api.elastic-cloud.com/api/v1/serverless/projects/elasticsearch"

project_data = {"name": "Serverless Search Project", "region_id": "aws-us-east-1"}

auth_header = f"ApiKey {api_key}"
headers = {"Content-Type": "application/json", "Authorization": auth_header}

es_project = requests.post(url, json=project_data, headers=headers)

Serverless projects generally start pretty quickly<br>We'll loop and wait for the project to be configured and ready to use

In [ ]:
if 200 <= es_project.status_code < 300:
    es_project_keys = es_project.json()
    prg_name = es_project_keys["name"]
    print(f"Project {prg_name} creation started")

    # wait for the project to be initialized and ready
    project_id = es_project.json()["id"]
    print("Checking if project is created and ready")
    loop = 1
    while True:
        es_project_check = requests.get(url + f"/{project_id}/status", headers=headers)
        if es_project_check.json()["phase"] == "initialized":
            break
        else:
            clear_output(wait=True)
            print(
                f"Waiting for project to be ready. Current status:{es_project_check.json()['phase']} - Loop {loop} Sleeping 10 seconds"
            )
            sleep(10)
            loop += 1

    print("Project is ready")

else:
    print(es_project.text)

Waiting for project to be ready. Current status:initializing - Loop 16 Sleeping 10 seconds
Project is ready


# Retrieve Serverless Project Connection Information

## Create elasticsearch client

We set the connection information from the `es_project` response above.

In [ ]:
es = Elasticsearch(
    es_project_keys["endpoints"]["elasticsearch"],
    basic_auth=(
        es_project_keys["credentials"]["username"],
        es_project_keys["credentials"]["password"],
    ),
)

## Project API Key
Create a [Project level API key](https://www.elastic.co/guide/en/elasticsearch/reference/current/security-api-create-api-key.html)

In [ ]:
project_key_response = es.security.create_api_key(
    name="full_access_key",
    metadata={"description": "API key for full access"},
    expiration="14d",
)

project_api_key = project_key_response["encoded"]
print(f"{project_key_response['name']} has been created")

full_access_key has been created


# Summary

In [ ]:
print("\n--- Project Summary ---")
print(f"Project Name: {es_project_keys['name']}")
print(f"Elasticsearch Endpoint: {es_project_keys['endpoints']['elasticsearch']}")
if "kibana" in es_project_keys["endpoints"]:
    print(f"Kibana Endpoint: {es_project_keys['endpoints']['kibana']}")
else:
    print(
        "Kibana Endpoint: Not available (Kibana may not be part of the serverless project)"
    )


print("\n--- API Key ---")
print(f"API key stored in variable: project_api_key")

show_api_key = input(
    "Do you want to print the API key in clear text? (yes/no): "
).lower()

if show_api_key == "yes":
    print(f"Project API Key: {project_api_key}")
else:
    print("API key not printed in clear text as requested. Stored in `project_api_key`")